In [ ]:
import numpy as np
import cv2

In [ ]:
import cv2
import numpy as np

def overlay_image(background, overlay, x, y):
    """
    Overlay an image onto another at the specified coordinates.

    Args:
    - background (np.array): The background image.
    - overlay (np.array): The overlay image.
    - x (int): The x coordinate on the background where the overlay should be placed.
    - y (int): The y coordinate on the background where the overlay should be placed.
    """
    # Get the dimensions of the overlay
    h, w = overlay.shape[:2]

    # Check if the overlay goes beyond the bounds of the background
    if x + w > background.shape[1] or y + h > background.shape[0]:
        raise ValueError("The overlay image extends beyond the background image at the specified coordinates.")

    # If the overlay image has an alpha channel
    if overlay.shape[2] == 4:
        alpha_overlay = overlay[:, :, 3] / 255.0
        alpha_background = 1.0 - alpha_overlay

        # Loop over all channels (assuming BGR format)
        for c in range(0, 3):
            background[y:y+h, x:x+w, c] = (alpha_overlay * overlay[:, :, c] +
                                           alpha_background * background[y:y+h, x:x+w, c])
    else:
        # If no alpha channel, simply overlay the image on top
        background[y:y+h, x:x+w] = overlay

    return background



In [ ]:
import cv2
import numpy as np
import math

def rotate_image(image, angle, pivot):
    """
    Rotate an image around a pivot point.
    """
    # Get the image size
    h, w = image.shape[:2]

    # Calculate the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(pivot, angle, 1.0)

    # Perform the rotation
    rotated = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR)

    return rotated

def rotate_around_center(background, overlay, angle_step=1, writeout=False):
    """
    Rotate the overlay image around the center of the background image.
    """
    # Calculate the center of the background image
    bg_h, bg_w = background.shape[:2]
    bg_center = (bg_w // 2, bg_h // 2)

    # The radius from the center where the overlay image will be placed
    radius = min(bg_w, bg_h) // 3

    directions = [1.0, -1.0, -1.0, 1.0]
    
    frame_num = 0
    while(True):
        angle = 270
        for dir in directions:
            for _ in range(0, 45, angle_step):
                # Make a copy of the background to avoid drawing over it multiple times
                bg_copy = background.copy()

                # Calculate the overlay's top-left corner for the current angle
                radians = np.deg2rad(angle)
                x = int(bg_center[0] + radius * math.cos(radians) - overlay.shape[1] / 2)
                y = int(bg_center[1] + radius * math.sin(radians) - overlay.shape[0] / 2)

                # Rotate the overlay image
                rotated_overlay = rotate_image(overlay, -angle, (overlay.shape[1] // 2, overlay.shape[0] // 2))

                # Use the previously defined `overlay_image` function here to combine `bg_copy` and `rotated_overlay`
                result = overlay_image(bg_copy, rotated_overlay, x, y)

                # Display the result
                cv2.imshow('Rotating Overlay', result)
                cv2.waitKey(10)  # Display each frame for 100 milliseconds

                # Optionally, save each frame to a file
                if(writeout):
                    # Remove alpha channel
                    result = cv2.resize(result[..., 0:3], (400, 400))
                    cv2.imwrite('./rotating_speaker/frame_{0:04d}.jpg'.format(frame_num), result)

                angle += dir
                frame_num += 1
                print('Frame number: ', frame_num)
        break

    cv2.destroyAllWindows()

# Load your images (adjust the paths as necessary)
background_img = cv2.imread('head_and_ears.png')
overlay_img = cv2.imread('speaker2.png', cv2.IMREAD_UNCHANGED)  # Assuming overlay has transparency

# Call the function to start rotating the overlay image
rotate_around_center(background_img, overlay_img, writeout=True)


In [ ]:
# Load your images (replace 'path/to' with the actual paths)
background_img = cv2.imread('./head_and_ears.png')
overlay_img = cv2.imread('./speaker.png', cv2.IMREAD_UNCHANGED)  # Load with alpha channel if present

# Specify the top-left coordinates for the overlay (for example, (50, 100))
x, y = 50, 100

# Overlay the image
result = overlay_image(background_img, overlay_img, x, y)

# Display the result
cv2.imshow('Overlay Result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Optionally, save the result to a file
#cv2.imwrite('path/to/result.jpg', result)

In [ ]:
np.arange(10)[0:3]

In [ ]:
# Now we combine the images
from PIL import Image
import os

def find_images_in_directory(directory):
    """
    Returns a list of image paths found in the specified directory.
    Supports common image formats: .jpg, .jpeg, .png, .bmp, .gif.
    """
    img_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
    return [os.path.join(directory, f) for f in os.listdir(directory) if os.path.splitext(f)[1].lower() in img_extensions]

def combine_images(img_path1, img_path2):
    """
    Combines two images side by side and returns the combined image.
    """
    image1 = Image.open(img_path1)
    image2 = Image.open(img_path2)

    # Calculate dimensions for the combined image
    max_width = max(image1.width, image2.width)
    max_height = max(image1.height, image2.height)
    total_width = 2*max_width

    image1 = image1.resize((max_width, max_height))
    image2 = image2.resize((max_width, max_height))
    # Create a new image with appropriate dimensions
    new_img = Image.new('RGB', (total_width, max_height))

    # Paste the two images into the new image
    new_img.paste(image1, (0, 0))
    new_img.paste(image2, (image1.width, 0))

    return new_img

def main(directory1, directory2, output_directory):
    """
    Combines images from two directories and saves them to an output directory.
    """
    # Find images in both directories
    dir1_images = sorted(find_images_in_directory(directory1))
    dir2_images = sorted(find_images_in_directory(directory2))

    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Iterate over the list of image paths, combining and saving them
    for i, (img_path1, img_path2) in enumerate(zip(dir1_images, dir2_images), start=1):
        combined_img = combine_images(img_path1, img_path2)
        output_path = os.path.join(output_directory, 'combined_{0:04d}.jpg'.format(i))
        combined_img.save(output_path)
        print(img_path1, img_path2)
        #print(f'Saved: {output_path}')

if __name__ == '__main__':
    directory1 = '../ears/'
    directory2 = './rotating_speaker/'
    output_directory = './combined_ears_and_speaker/'
    main(directory1, directory2, output_directory)

In [ ]:
os.getcwd()